Trimming and merging reads from .ab1 Sanger sequencing files.

## Quality trimming

* Isolate segment with the highest quality from the sequence, using Kadane's algorithm.
* Trim low-quality ends of the sequence.

## [<svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M7.775 3.275a.75.75 0 001.06 1.06l1.25-1.25a2 2 0 112.83 2.83l-2.5 2.5a2 2 0 01-2.83 0 .75.75 0 00-1.06 1.06 3.5 3.5 0 004.95 0l2.5-2.5a3.5 3.5 0 00-4.95-4.95l-1.25 1.25zm-4.69 9.64a2 2 0 010-2.83l2.5-2.5a2 2 0 012.83 0 .75.75 0 001.06-1.06 3.5 3.5 0 00-4.95 0l-2.5 2.5a3.5 3.5 0 004.95 4.95l1.25-1.25a.75.75 0 00-1.06-1.06l-1.25 1.25a2 2 0 01-2.83 0z"></path></svg>](https://github.com/sam-k/seq-quality-trimming/branches#merging-reads)Merging reads

* Align the forward/reverse PCR products using the Smith-Waterman algorithm for local alignment.
* Merge the reads if the overlap is large enough.

## [<svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path fill-rule="evenodd" d="M7.775 3.275a.75.75 0 001.06 1.06l1.25-1.25a2 2 0 112.83 2.83l-2.5 2.5a2 2 0 01-2.83 0 .75.75 0 00-1.06 1.06 3.5 3.5 0 004.95 0l2.5-2.5a3.5 3.5 0 00-4.95-4.95l-1.25 1.25zm-4.69 9.64a2 2 0 010-2.83l2.5-2.5a2 2 0 012.83 0 .75.75 0 001.06-1.06 3.5 3.5 0 00-4.95 0l-2.5 2.5a3.5 3.5 0 004.95 4.95l1.25-1.25a.75.75 0 00-1.06-1.06l-1.25 1.25a2 2 0 01-2.83 0z"></path></svg>](https://github.com/sam-k/seq-quality-trimming/branches#blast)BLAST

* BLAST the merged sequence.

https://github.com/sam-k/seq-quality-trimming

http://fanhuan.github.io/en/2017/07/13/Batch-process-forward-reverse-16s/

In [2]:
import numpy as np
import os

# Biopython
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [81]:
data = "/mnt/d/Lab/TaxaIdentification/data/16S-seq-result"

In [82]:
filenames = []
Flist = []
Rlist = []

In [83]:
for file in os.listdir(data):
    if file.endswith("_[27F].ab1") and os.path.isfile(os.path.join(data, file)):
        filename = file.split("_")[2].replace("(", "").replace(")", "")
        filenames.append(filename)
        #Fpath = os.path.join(data, file)
        #Rpath = os.path.join(data, file[:-10]+"_[1492R].ab1")

In [84]:
for i in range(len(filenames)):
    for file in os.listdir(data):
        if filenames[i] in file and file.endswith(".ab1"):
            if file.endswith("_[27F].ab1"):
                Flist.append(os.path.join(data, file))
            elif file.endswith("_[1492R].ab1"):
                Rlist.append(os.path.join(data, file))

In [85]:
len(filenames)

71

In [86]:
71*4

284

In [87]:
Flist[0:5]

['/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0001_31320080702720_(B11)_[27F].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0003_31320080702721_(B12)_[27F].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0007_31320080702723_(B14)_[27F].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0009_31320080702724_(B15)_[27F].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0011_31320080702725_(B16)_[27F].ab1']

In [88]:
Rlist[0:5]

['/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0002_31320080702720_(B11)_[1492R].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0004_31320080702721_(B12)_[1492R].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0008_31320080702723_(B14)_[1492R].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0010_31320080702724_(B15)_[1492R].ab1',
 '/mnt/d/Lab/TaxaIdentification/data/16S-seq-result/0012_31320080702725_(B16)_[1492R].ab1']

In [89]:
DNA_COMP = {"A":"T", "T":"A", "C":"G", "G":"C", "N":"N"}
def rev_cmp(seq):
    revcmp_seq = ""
    for c in reversed(seq):
        revcmp_seq += DNA_COMP[c]
    return revcmp_seq


In [90]:
def import_seqs(filenames, Flist, Rlist):
    sequences = []
    for i in range(len(filenames)):
        names = (filenames[i]+"_F", filenames[i]+"_R")
        reads = (SeqIO.read(Flist[i], "abi"),
                 SeqIO.read(Rlist[i], "abi"))
        seqs  = (str(reads[0].seq), rev_cmp(str(reads[1].seq)))
        quals = ([x for x in reads[0].letter_annotations["phred_quality"]],
                  [x for x in reads[1].letter_annotations["phred_quality"]][::-1])
        sequences.append({"names":names, "seqs":seqs, "quals":quals})
    return sequences

In [91]:
import_seqs(filenames, Flist, Rlist)

[{'names': ('B11_F', 'B11_R'),
  'seqs': ('GGGGGGGGGGTGCTACACATGCAGTCGAACGATTAAGACGGCTCCGGCCGTGAATAGAGTGGCGAACGGGTGAGTAACACGTGACCAACCTGCCCCCTACCCCGGGATAACGCAAGGAAACCTGCGCTAATACCGGATACTCCTTCCCCTGCTCCTGCAGGGGTCGGGAAAGCCCAGACGGTAGGGGATGGGGTCGCGGCCCATTAGGTAGACGGCGGGGCAACGGCCCACCGTGCCTGCGATGGGTAGCCGGGTTGAGAGACCGACCGGCCACATTGGGACTGAGATACGGCCCAGACTCCTACGGGAGGCAGCAGTGGGGAATTTTGCGCAATGGGGGGAACCCTGACGCAGCAACGCCGCGTGCGGGATGAAGGCCTTCGGGTTGTAAACCGCTTTCAGCAGGGAAGATAGTGACGGTACCTGCAGAAGAAGCCCCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGGGCGAGCGTTATCCGGATTCATTGGGCGTAAAGCGCGCGTAGGCGGCCCGTCAAGCGGAACCTCTAACCCGAGGGCTCAACCCCCGGTCGGGTTCCGAACTGGCAGGCTCGAGTTTGGTAGAGGAAGATGGAATTCCCGGTGTAGCGGTGGAATGCGCAGATATCGGGAAGAACACCGATGGCGAAGGCAGTCTTCTGGGCCATCAACTGACGCTGAGGCGCGAAAGCTGGGGGAGCGAACAGGATTAGATACCCTGGTAGTCCCAGCCGTAAACGATGGGTGCTAGGTGTGGGGGGATCATCCCTCCGTGCCGCAGCCAACGCATTAAGCACCCCGCCTGGGGAGTACGGCCGCAAGGCTAAAACTCAAAGGAATTGACGGGGGCCCGCACAAGCAGCGGAGCATGTGGCTTAATTCGAAGCAACGCGAAGAACCTTACCAGGGCTTGACATGCTAGTGAAGCCGGGGAAACCCGGTGG